In this notebook we define the BZ map and its associated constants.

The BZ map was introduced in [MT], it is a $C^1$ map on $[0, 1]$
piecewise defined.

We need to compute some constants to guarantee continuity of the derivative and some dynamical properties.

On $[0, \frac{1}{8}]$ the map is defined by 
$$
T_{a,b}(x)=\left(a-\left(\frac{1}{8}-x\right)^{\frac{1}{3}} \right)e^{-x}+b
$$

On $[\frac{1}{8}, 0.3]$ the map is defined by
$$
T_{a,b}(x)=\left(a+\left(x-\frac{1}{8}\right)^{\frac{1}{3}} \right)e^{-x}+b
$$

On $[0.3, 1]$ it is defined by 
$$
T_{b,c}(x)= c\left(10x e^{\frac{-10x}{3}}\right)^{19}+b.
$$

We first compute $a, c$ such that $T$ and $T'$ are continuous
in $0.3$.

In [ ]:
import Pkg; Pkg.activate("../")

In [ ]:
using Symbolics, IntervalArithmetic
@variables a, b, c, x

setprecision(BigFloat, 1024)

expr_T_l = (a+(x-1//8)^(1//3))*exp(-x)+b

In [ ]:
expr_T_r = c*(BigInt(10)*x*exp(-Interval{BigFloat}(10)/3*x))^(19)+b

In [ ]:
val_l = substitute(expr_T_l, (Dict(x=> Interval{BigFloat}(3)/10)))

In [ ]:
val_r = substitute(expr_T_r, (Dict(x=> Interval{BigFloat}(3)/10)))

In [ ]:
eq_val = Equation(val_r, val_l)

In [ ]:
D = Differential(x)

der_l = expand_derivatives(D(expr_T_l))

In [ ]:
der_val_l = substitute(der_l, (Dict(x=> Interval{BigFloat}(3)/10)))

In [ ]:
der_r = expand_derivatives(D(expr_T_r))

In [ ]:
der_val_r = substitute(der_r, (Dict(x=> Interval{BigFloat}(3)/10)))

In [ ]:
der_eq = Equation(der_val_l, 0)

In [ ]:
z = Symbolics.solve_for(der_eq, a)

In [ ]:
typeof(z)
Symbolics.value(z)

In [ ]:
der_l_func = Symbolics.build_function(der_l, a, x, expression = Val(false))

In [ ]:
using IntervalArithmetic

val_der_l = der_l_func(a, Interval{BigFloat}(3)/10)
val_der_func(a) = der_l_func(a, Interval{BigFloat}(3)/10)

In [ ]:
using DualNumbers
val_der_der_a(a) = val_der_func(Dual(a, 1)).epsilon

In [ ]:
function root(f, f′, x, ϵ; max_iter = 100)
	for i in 1:max_iter
		x_old = x
		x_mid = Interval(mid(x))
		x = intersect(x, x_mid - f′(x) \ f(x_mid))
		if x_old == x || isempty(x) || diam(x) < ϵ
			return x
		end
	end
	@info "Maximum iterates reached" max_iter, x, f(x)
	return x
end

In [ ]:
setprecision(BigFloat, 1024)
A = root(val_der_func, val_der_der_a, Interval{BigFloat}(0,1), 10^(-30))

In [ ]:
using IntervalArithmetic
A, diam(A)

In [ ]:
sub_eq_val_A = substitute(eq_val, Dict(a=>A))

In [ ]:
z = Symbolics.solve_for(sub_eq_val_A, c)

In [ ]:
C = Symbolics.value(z)

In [ ]:
C, diam(C)

We have now found the value of $A$ and $C$

In [ ]:
T_left_leq_1_8(x, b) = (A-abs(Interval{BigFloat}(1)/8-x)^(Interval{BigFloat}(1)/3))*exp(-x)+b

In [ ]:
using Plots

In [ ]:
plot(x->mid(T_left_leq_1_8(x, 0.023288528303070)), 0, 1//8)

In [ ]:
T_left_geq_1_8(x, b) = (A+abs(Interval{BigFloat}(1)/8-x)^(Interval{BigFloat}(1)/3))*exp(-x)+b

In [ ]:
plot!(x->mid(T_left_geq_1_8(x, 0.023288528303070)), 1//8, 0.3)

In [ ]:
T_right(x, b) =C*(BigInt(10)*x*exp(-Interval{BigFloat}(10)/3*x))^(19)+b

In [ ]:
plot!(x->mid(T_right(x, 0.023288528303070)), 0.3, 1)

We want to certify now the value of $b$.

In [ ]:
function T(x, b)
    if 0<=x<= 1//8
        return T_left_leq_1_8(x, b)
    elseif 1//8<x<0.3
        return T_left_geq_1_8(x, b)
    elseif x>=0.3
        return T_right(x, b)
    end
end

In [ ]:
plot(x->mid(T(x, 0.023288528303070)), 0, 1)

We start with an educated guess, i.e., $b = 0.023288528303070$.

In [ ]:
zero_three = @biginterval 0.3

In [ ]:
F(v) = [T(zero_three, v[1])-v[2]; T(v[2], v[1])-v[3]; T(v[3],v[1])-v[4]; T(v[4], v[1])-v[5]; T(v[5], v[1])-v[6]; T(v[6], v[1])-v[6]]

In [ ]:
B = @biginterval 0.02328852830307032054478158044023918735669943648088852646123182739831022528
B += 2^(-200)*Interval{BigFloat}(-1,1)
B, diam(B)

In [ ]:
x_test = @biginterval 0.4
diam(T_right(x_test, B))

In [ ]:
x1 = T(zero_three, B)
x2 = T(x1, B)
x3 = T(x2, B)
x4 = T(x3, B)
x5 = T(x4, B)

v = [x1; x2; x3; x4; x5]
v, maximum(diam.(v))

In [ ]:
F([B; v])

The vector $[b; v]$ is the starting point for our multivariate Newton Method.


In [ ]:
der_r_fixed_c = substitute(der_r, (Dict(c => C)))
func_der_r = Symbolics.build_function(der_r_fixed_c, x, expression = Val(false))
expr_T_l_leq_1_8 = (A-(Interval{BigFloat}(1)/8-x)^(Interval{BigFloat}(1)/3))*exp(-x)+b
der_expr_T_l_leq_1_8 = expand_derivatives(D(expr_T_l_leq_1_8))
der_T_l_leq_1_8_func = Symbolics.build_function(der_expr_T_l_leq_1_8, x, expression = Val(false))
expr_T_l_geq_1_8 = (A+(x-Interval{BigFloat}(1)/8)^(Interval{BigFloat}(1)/3))*exp(-x)+b
der_expr_T_l_geq_1_8 = expand_derivatives(D(expr_T_l_geq_1_8))
der_T_l_geq_1_8_func = Symbolics.build_function(der_expr_T_l_geq_1_8, x, expression = Val(false))

function der_T(x)
    if 0<=x<= 1//8
        return der_T_l_leq_1_8_func(x)
    elseif 1//8<x<0.3
        return der_T_l_geq_1_8_func(x)
    elseif x>=0.3
        return func_der_r(x)
    end
end

In [ ]:
diam(der_T(Interval{BigFloat}(1//8)))

In [ ]:
function Jac(v)
    n = length(v)
    J = zeros(Interval{BigFloat}, (n,n))
    J[:, 1] = ones(BigFloat, n)
    for i in 1:(n-1)
        J[i, i+1] = -1
    end
    J[n, n]= -1

    for i in 2:n
        J[i, i] += der_T(v[i])
    end

    return J
end

In [ ]:
w = [B; v]
J = Jac(w)

In [ ]:
maximum(diam.(Jac(w)))

In [ ]:
w_next = mid.(w)-inv(Jac(w))*F(Interval{BigFloat}.(mid.(w)))
w = w_next
w, maximum(diam.(w))

In [ ]:
using JLD2

In [ ]:
BZcoeff = Dict("a"=>A, "b"=> B, "c"=> C)
save("bzmapcoeff.jld2", BZcoeff)

In [ ]:
using Plots

In [ ]:
plot(x->mid(der_T(x)), 0, 1)